<a href="https://colab.research.google.com/github/Filippo-Tombari/PdeGraph/blob/main/Obstacle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Risolviamo numericamente l'equazione

\begin{equation}
\frac{\partial u}{\partial t} - \Delta u + \mathbf{b}\cdot\nabla u= 0
\end{equation}

dove $\mathbf{b}=[0,1]$. Al bordo consideriamo condizioni di Neumann omogenee. Usiamo

\begin{equation}
u_{0}(x,y) = (x+1)^{2} + (y+1)^{2},
\end{equation}

come profilo iniziale.

Applicando il metodo di Eulero all'indietro in tempo abbiamo

\begin{equation}
\frac{u^{n+1} - u^{n}}{\Delta t} - \Delta u^{n+1} + \mathbf{b}\nabla u^{n+1} = 0,
\end{equation}

da cui

\begin{equation}
\implies u^{n+1} - \Delta t\Delta u^{n+1} + \Delta t\mathbf{b}\nabla u^{n+1} = u^{n}.
\end{equation}


# Import the necessary packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/tesi/PdeGraph")
import os
os.chdir("/content/drive/MyDrive/tesi/PdeGraph")

Mounted at /content/drive


In [2]:
import install
install.pytorchgeo()

Pytorch geometric installed.


In [3]:
install.fenics()

FEniCS installed.


In [4]:
import numpy as np
import pickle
import functional
from functional import asfield, plot, L2, buildconnectivity
import gnns
import dolfin
import torch
import torch.nn.functional as F
from torch_geometric.loader import NeighborSampler
import torch.optim as optim

# Loading and preparation of the data

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
mesh_train = dolfin.cpp.mesh.Mesh("files/geometry_coarse.xml")
mesh_test = dolfin.cpp.mesh.Mesh("files/geometry_test.xml")
#create edge connectivity matrices
edge_index_train = buildconnectivity(mesh_train)
edge_index_train = torch.t(torch.from_numpy(edge_index_train.astype('int32')).long())
edge_index_test = buildconnectivity(mesh_test)
edge_index_test = torch.t(torch.from_numpy(edge_index_test.astype('int32')).long())
#create festures matrices
fts_train = torch.from_numpy(np.load("files/traiettoria_coarse.npy")).float().unsqueeze(dim = 2).to(device)
fts_test = torch.from_numpy(np.load("files/traiettoria_test.npy")).float().unsqueeze(dim = 2).to(device)

print("training edge connectivity matrix shape: ", edge_index_train.shape)
print("test edge connectivity matrix shape: ", edge_index_test.shape)
print("training feature matrix shape:", fts_train.shape)
print("test feature matrix shape:", fts_test.shape)

training edge connectivity matrix shape:  torch.Size([2, 1939])
test edge connectivity matrix shape:  torch.Size([2, 28536])
training feature matrix shape: torch.Size([11, 693, 1])
test feature matrix shape: torch.Size([11, 9698, 1])


In [39]:
#create training, validation and test set 
torch.manual_seed(0)
valid = False
if valid == False:
  train_loader = NeighborSampler(edge_index_train, node_idx=None,
                               sizes=[8, 7, 3], batch_size=fts_train.size()[1])
else:
  train_size = int(fts_train.size()[0]*0.8)
  valid_size = fts_train.size()[0] - train_size
  indices = [x for x in range(fts_train.size()[0])]
  train_idx, valid_idx = train_test_split(indices,train_size = train_size)
  train_loader = NeighborSampler(edge_index_train, node_idx=torch.Tensor(train_idx).long(),
                                sizes=[8, 7, 3], batch_size=train_size)
  valid_loader = NeighborSampler(edge_index_train, node_idx=torch.Tensor(valid_idx).long(),
                                sizes=[-1], batch_size=valid_size)
  
subgraph_loader = NeighborSampler(edge_index_test, node_idx=torch.Tensor(range(fts_test.size()[0])).long(),
                                  sizes=[-1], batch_size=fts_test.size()[0])

In [40]:
for bs, id, a in train_loader:
  batch_size_train = bs
  n_id_train = id
  adjs_train = a #`adjs` holds a list of `(edge_index, e_id, size)` tuples
if valid == True:
  for bs, id, a in valid_loader:
    batch_size_valid = bs
    n_id_valid = id
    adjs_valid = a #`adjs` holds a list of `(edge_index, e_id, size)` tuples

In [13]:
for u in fts_train[:]:
  print(u.shape)

torch.Size([693, 1])
torch.Size([693, 1])
torch.Size([693, 1])
torch.Size([693, 1])
torch.Size([693, 1])
torch.Size([693, 1])
torch.Size([693, 1])
torch.Size([693, 1])
torch.Size([693, 1])
torch.Size([693, 1])
torch.Size([693, 1])


# Training

In [16]:
l2   = L2(mesh_train).float() # L2 norm for scalar functions
lv22 = lambda v: l2(v[:,:,0].to(device)).pow(2).float() #+ l2(v[:,:,1].to(device)).pow(2).float()
lv2  = lambda v: lv22(v).sqrt().float() # L2 norm for vectorial functions
def loss(output, target):
  return (lv2(target - output) / lv2(target)).mean().float()

In [55]:
gnn = gnns.SAGE(1,256,1).to(device)
learningrate = 1e-2
optimizer = optim.Adam(gnn.parameters(), lr=learningrate)
#optimizer = optim.LBFGS(gnn.parameters())
model_chk_path = 'checkpoints/basic_chk.pt'
mse_min = 10000
early_stopping = 0
epochs = 500 
t = 1 # current epoch
done = False
dt = 2e-2
while not done:      
      train_loss = 0
      # training
      def closure():
        optimizer.zero_grad()
        # forward pass
        integrating = torch.stack([gnn.forward(u, adjs_train, device) for u in fts_train[:]], axis = 0)
        train_out = (fts_train[[0]] + dt*integrating.cumsum(axis = 0)) #u(t1) = u(t0) + int{t0,t1}phi(t)dt
        train_loss = loss(train_out[:-1],fts_train[1:])
        # backpropagation
        #torch.autograd.set_detect_anomaly(True)
        train_loss.backward()
        return train_loss
      optimizer.step(closure)
      with torch.no_grad():
        integrating = torch.stack([gnn.forward(u, adjs_train, device) for u in fts_train[:]], axis = 0)
        train_out = (fts_train[[0]] + dt*integrating.cumsum(axis = 0)) #u(t1) = u(t0) + int{t0,t1}phi(t)dt
        train_loss = loss(train_out[:-1],fts_train[1:])
        #integrating_valid = torch.stack([gnn.inference(u, valid_loader, device) for u in valid[0:]], axis = 0)
        #valid_out = valid[0] + dt*integrating_valid.cumsum(axis = 0)
        #valid_loss = loss(valid_out[:-1],valid[1:])

      # print rollout number and MSE for training and validation set at each epoch
      print(f"Rollout {t:1f}: MSE_train {train_loss :6.3f}" )
      if train_loss < mse_min:
        mse_min = train_loss
        train_out_best = train_out
        torch.save(gnn, model_chk_path)
        early_stopping = 0
        print('Saving model checkpoint')
      else:
        early_stopping += 1
      #stop the training after reaching the number of epochs
      t += 1
      #import pdb; pdb.set_trace()
      if (t > epochs ): #or early_stopping == 20
        done = True

Rollout 1.000000: MSE_train  0.318
Saving model checkpoint
Rollout 2.000000: MSE_train  0.315
Saving model checkpoint
Rollout 3.000000: MSE_train  0.312
Saving model checkpoint
Rollout 4.000000: MSE_train  0.310
Saving model checkpoint
Rollout 5.000000: MSE_train  0.308
Saving model checkpoint
Rollout 6.000000: MSE_train  0.307
Saving model checkpoint
Rollout 7.000000: MSE_train  0.305
Saving model checkpoint
Rollout 8.000000: MSE_train  0.303
Saving model checkpoint
Rollout 9.000000: MSE_train  0.302
Saving model checkpoint
Rollout 10.000000: MSE_train  0.301
Saving model checkpoint
Rollout 11.000000: MSE_train  0.299
Saving model checkpoint
Rollout 12.000000: MSE_train  0.298
Saving model checkpoint
Rollout 13.000000: MSE_train  0.297
Saving model checkpoint
Rollout 14.000000: MSE_train  0.296
Saving model checkpoint
Rollout 15.000000: MSE_train  0.294
Saving model checkpoint
Rollout 16.000000: MSE_train  0.292
Saving model checkpoint
Rollout 17.000000: MSE_train  0.289
Saving model 

In [56]:
test_loader = NeighborSampler(edge_index_test, node_idx=None,
                               sizes=[-1], batch_size=fts_test.size()[1])

In [31]:
def forecast(u0, model, steps):
  res = [u0]
  with torch.no_grad():
    for i in range(steps):
      res.append(res[-1]+dt*model.inference(res[-1],test_loader,device))
  return torch.stack(res, axis = 0)

In [57]:
gnn.inference(fts_test[0],test_loader,device).shape

torch.Size([9698, 1])

In [60]:
U = forecast(fts_test[0], gnn, steps = 10)

In [62]:
l2   = L2(mesh_test).float() # L2 norm for scalar functions
lv22 = lambda v: l2(v[:,:,0].to(device)).pow(2).float() #+ l2(v[:,:,1].to(device)).pow(2).float()
lv2  = lambda v: lv22(v).sqrt().float() # L2 norm for vectorial functions
def loss(output, target):
  return (lv2(target - output) / lv2(target)).mean().float()

In [63]:
loss(U,fts_test)

tensor(0.0614, device='cuda:0')

In [52]:
# Righe di codice per salvare l'animazione in formato .gif
import imageio
import matplotlib.pyplot as plt
def savegif(drawframe, frames, name, transparency = False, remove = True):
    filenames = []
    for i in range(frames):
        # plot frame
        drawframe(i)

        # create file name and append it to a list
        filename = f'{i}.png'
        filenames.append(filename)

        # save frame
        plt.savefig(filename, transparency = transparency)
        plt.close()
    # build gif
    with imageio.get_writer(name + '.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove files
    if(remove):
        for filename in set(filenames):
            os.remove(filename)

def trajectorytogif(traj, dt, name):
    def drawframe(i):
        colorbar = plot(functional.asfunction(traj[i,:,0], mesh_test))
        plt.colorbar(colorbar, shrink = 0.75)
        plt.title("T = %.2f" % (dt*i))
        plt.axis("off")
    savegif(drawframe, frames = len(traj), name = name)

In [64]:
trajectorytogif(U, 1, name = "images/test_basic_example_pred") # crea e salva la gif (la si trova nella cartella dei file generati, a sx del notebook)

# Nota: su Colab non si può, ma su jupyter notebook è invece possibile visualizzare poi la gif direttamente
# dentro il notebook, e.g.

# from  IPython.display import Image as show
# show("esempio.gif")

In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Obstacle.ipynb
	modified:   gnns.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	__pycache__/
	checkpoints/obstacle_chk.pt

no changes added to commit (use "git add" and/or "git commit -a")


In [65]:
!git config --global user.email "filo.tombari@gmail.com"
!git config --global user.name "Filippo-Tombari"

In [ ]:
!git add .
!git commit -m "training con gpu e lbfgs"
!git push -u origin main

[main cdf140a] training con gpu e lbfgs
 6 files changed, 5 insertions(+), 3 deletions(-)
 rewrite Obstacle.ipynb (98%)
 create mode 100644 __pycache__/functional.cpython-37.pyc
 create mode 100644 __pycache__/gnns.cpython-37.pyc
 create mode 100644 __pycache__/install.cpython-37.pyc
 create mode 100644 checkpoints/obstacle_chk.pt
Counting objects: 10, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (10/10), done.
Writing objects: 100% (10/10), 505.71 KiB | 12.97 MiB/s, done.
Total 10 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/Filippo-Tombari/PdeGraph.git
   e76a7c6..cdf140a  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
